In [1]:
import csv
import os
import json
import codecs
import re
import pandas as pd

In [2]:
import xlrd

In [3]:
eng_data = pd.read_excel('english.xlsx')

In [4]:
eng_data.dropna(inplace=True)

In [5]:
data = list(eng_data['text'])

In [6]:
import numpy as np
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [8]:
new_data = []

for i in range (len(data)):
    clean_tweet = re.match('(.*?)http.*?\s?(.*?)', data[i])
    if clean_tweet:
        new_tweet = ''+clean_tweet.group(1) + clean_tweet.group(2)
        
        new_data.append(new_tweet)
    else:
        new_data.append(data[i])

data = new_data

In [9]:
eng_data['text'] = data

In [10]:
eng_data['datetime'] = pd.to_datetime(eng_data['datetime'])

eng_data = eng_data.sort_values('datetime')

In [11]:
eng_data.drop(eng_data.columns[0],axis=1,inplace=True)

In [12]:
eng_data.index = list(np.arange(len(eng_data)))

In [13]:
eng_data

,usernameTweet,ID,text,url,nbr_retweet,nbr_favorite,nbr_reply,datetime,has_media,medias,is_reply,is_retweet,user_id
0,DOAWAYWDENGUE,-1.807863e+09,# DENGUEFEVER #AWARENESS - `Dengue may break...,/DOAWAYWDENGUE/status/19666973140,0.0,0.0,0.0,2010-07-27 08:54:07,False,false,False,False,3.354970e+07
1,Crof,5.747630e+08,Bangladesh : 473 #dengue cases in Dhaka,/Crof/status/102929918320586753,0.0,0.0,0.0,2011-08-14 19:29:39,False,false,False,False,1.904299e+07
2,DengueNews,-2.682962e+08,# Dengue News: Eradicating dengue menace - Fi...,/DengueNews/status/110057252496482304,0.0,0.0,0.0,2011-09-03 11:31:08,False,false,False,False,3.640113e+08
3,Mazedul,-7.115981e+07,"Urgent O- ( O Negative) Blood Needed \n""O-"" ...",/Mazedul/status/137597059736940544,0.0,0.0,0.0,2011-11-18 10:24:30,False,false,False,False,3.761954e+07
4,DengueNews,1.484927e+09,# Dengue News: Dengue-like fever returns stro...,/DengueNews/status/139426347993935872,0.0,0.0,0.0,2011-11-23 11:33:26,False,false,False,False,3.640113e+08
5,DengueNews,-1.891484e+09,# Dengue News: Hydroclimatological variabilit...,/DengueNews/status/194786278334676993,0.0,0.0,0.0,2012-04-24 06:54:02,False,false,False,False,3.640113e+08
6,OnlyinDhaka,-1.048445e+09,"Ah, the smell of #Dhaka 's open #sewerage in ...",/OnlyinDhaka/status/268289421768523776,0.0,0.0,0.0,2012-11-13 01:49:17,False,false,False,False,9.075084e+08
7,selimbd08,1.170362e+09,"O+ Blood Needed, Loc- Lab-Aid Hospital, Dhan...",/selimbd08/status/271294269006958592,0.0,0.0,0.0,2012-11-21 08:49:28,False,false,False,False,3.521479e+08
8,IDRC_CRDI,5.496136e+08,« Notre recherche permettra d’établir des donn...,/IDRC_CRDI/status/291987055746117632,0.0,0.0,0.0,2013-01-17 11:15:13,False,false,False,False,4.006517e+07
9,IDRC_CRDI,5.579776e+08,« Que savons-nous de la #dengue à Dhaka ? T...,/IDRC_CRDI/status/292023550091595776,1.0,0.0,0.0,2013-01-17 13:40:14,False,false,False,False,4.006517e+07


In [14]:
data

['What are the risk factors for the presence of #dengue  vector mosquitoes? Determinants of their prevalence and larval site selection in  Dhaka ,  ',
 'Emergency (O-)  Blood  Needed for  Dengue  patient.\nPlace: Insaf Barakah Hospital, Moghbazar\nContact: 01911831932 — looking for O-  blood  donors in  Dhaka   https://www. facebook.com/10000024204126 6/posts/2282699395081440/ \xa0 …',
 'The threat of #denguefever  looms over as the monsoon progresses, hitting  Bangladesh  with higher levels of rainfall. Over 80% of mosquitoes hatching eggs in motionless water in Dhaka city could be #dengue -spreading #AedesAegypti ,  ',
 'Low socio-economic zones of  Dhaka  may have higher presence of #AedesAegypti  mosquitoes: New insights,  ',
 'Recent findings on risk factors for the presence of #dengue  vector mosquitoes in  Dhaka ,  ',
 '# IEDCR  reports 680 #dengue  cases so far  ',
 'New report on circulating #dengue  virus serotypes in  Bangladesh  from 2013 to 2016,  ',
 "Bangladesh 's climat

In [15]:
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['What are the risk factors for the presence of #dengue vector mosquitoes? '
 'Determinants of their prevalence and larval site selection in Dhaka , ']


In [16]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])


[['what', 'are', 'the', 'risk', 'factors', 'for', 'the', 'presence', 'of', 'dengue', 'vector', 'mosquitoes', 'determinants', 'of', 'their', 'prevalence', 'and', 'larval', 'site', 'selection', 'in', 'dhaka']]


In [17]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['what', 'are', 'the', 'risk', 'factors', 'for', 'the', 'presence', 'of', 'dengue', 'vector', 'mosquitoes', 'determinants', 'of', 'their', 'prevalence', 'and', 'larval', 'site', 'selection', 'in', 'dhaka']


In [18]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['risk', 'factor', 'presence', 'dengue', 'vector', 'mosquitoe', 'determinant', 'prevalence', 'larval', 'site', 'selection', 'dhaka']]


In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)]]


In [21]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [22]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.163*"aede" + 0.105*"find" + 0.043*"net" + 0.035*"south" + 0.026*"surge" + '
  '0.023*"larvae" + 0.022*"house" + 0.019*"almost" + 0.012*"north" + '
  '0.002*"newagebd"'),
 (1,
  '0.230*"country" + 0.123*"mosquito_borne" + 0.036*"cause" + 0.036*"severe" + '
  '0.018*"rainy" + 0.015*"flu" + 0.010*"decade" + 0.006*"disease" + '
  '0.003*"grow" + 0.002*"occur"'),
 (2,
  '0.060*"due" + 0.051*"struggle" + 0.049*"low" + 0.036*"climatechange" + '
  '0.035*"count" + 0.031*"kill" + 0.030*"ab" + 0.026*"urgently" + '
  '0.011*"september" + 0.008*"brother"'),
 (3,
  '0.208*"chikungunya" + 0.086*"control" + 0.078*"last" + 0.073*"rise" + '
  '0.054*"fight" + 0.043*"government" + 0.041*"see" + 0.017*"compare" + '
  '0.014*"treat" + 0.009*"dhakatribune"'),
 (4,
  '0.147*"death" + 0.072*"denguefever" + 0.068*"hit" + 0.029*"make" + '
  '0.022*"frontpage" + 0.013*"php" + 0.009*"stor" + 0.009*"y_fbid" + '
  '0.008*"let" + 0.000*"die"'),
 (5,
  '0.159*"need" + 0.054*"suffer" + 0.044*"old" + 0.027*"

In [23]:
data_lemmatized

[['risk',
  'factor',
  'presence',
  'dengue',
  'vector',
  'mosquitoe',
  'determinant',
  'prevalence',
  'larval',
  'site',
  'selection',
  'dhaka'],
 ['emergency',
  'blood_neede',
  'dengue',
  'patient',
  'place',
  'insaf',
  'barakah',
  'hospital',
  'moghbazar',
  'contact',
  'look',
  'blood_donors',
  'dhaka',
  'https',
  'www',
  'facebook',
  'com',
  'post'],
 ['threat',
  'denguefever',
  'loom',
  'monsoon',
  'progress',
  'hit',
  'bangladesh',
  'high',
  'level',
  'rainfall',
  'mosquitoe',
  'hatch',
  'egg',
  'motionless',
  'water',
  'dhaka',
  'city',
  'could',
  'dengue',
  'spread',
  'aedesaegypti'],
 ['low',
  'socio',
  'economic',
  'zone',
  'dhaka',
  'may',
  'high',
  'presence',
  'aedesaegypti',
  'mosquitoe',
  'new',
  'insight'],
 ['recent',
  'finding',
  'risk',
  'factor',
  'presence',
  'dengue',
  'vector',
  'mosquitoe',
  'dhaka'],
 ['iedcr', 'report', 'dengue', 'case', 'far'],
 ['new', 'report', 'circulate', 'dengue', 'virus',

In [24]:
cities = pd.read_csv('haha.csv' , sep = '\t')

In [25]:
cities

,Name of metropolitan thana and GO code,Area (sq km),Ward and Union,Mahalla and Mouza,Population,Density (per sq km),Literacy rate (%)
0,Adabor,2.07,2,8,86540,41806.00,62.24
1,Uttar Khan,20.09,1,14,52014,2590.00,64.58
2,Uttara,36.91,1,13,66636,11295.00,65.67
3,Kadamtali,10.16,4+2 (Part),15,330565,32536.00,63.87
4,Kalabagan,1.26,1+1 (Part),10,106671,84660.00,76.88
5,Kafrul,7.89,1+3 (Part),18,272939,34593.00,69.87
6,Kamrangirchar,3.63,1,12,143208,39452.00,42.84
7,Cantonment,14.47,1+1 (Part),8,117464,11393.00,70.30
8,Kotwali,0.67,2+2 (Part),44,82488,123117.00,77.78
9,Khilkhet,15.88,1,11,39227,2470.00,58.32


In [26]:
eng_lst_areas = cities['Name of metropolitan thana and GO code'].tolist()

In [27]:
eng_lst_areas.extend(['mirpur','azimpur','mugda','postogola','shyamoli','mohammadpur','moghbazar','maghbazar','mogbazar','chan khar pool','science laboratory','science lab','buet','du','dhaka university','dhaka medical','dmc','shantinagar','meradia','notun bazar','new paltan','noya paltan','technical','gabtoli','motijheel','karwanbazar','farmgate','khamarbari','tejkuni para','purobi','meradia','aftabnagar','aftab nagar','puran dhaka','shankar','dia bari','banani','rampura','sadarghat','gulistan','hatirpool','elephant road','tikatuli','airport','baily road','mintu road','zia colony','savar'])

In [28]:
eng_lst_areas

['Adabor',
 'Uttar Khan',
 'Uttara',
 'Kadamtali',
 'Kalabagan',
 'Kafrul',
 'Kamrangirchar',
 'Cantonment',
 'Kotwali',
 'Khilkhet',
 'Khilgaon',
 'Gulshan',
 'Gendaria',
 'Chawkbazar Model',
 'Demra',
 'Turag',
 'Tejgaon',
 'Tejgaon I/A',
 'Dakshinkhan',
 'Darus Salam',
 'Dhanmondi',
 'New Market',
 'Paltan',
 'Pallabi',
 'Bangshal',
 'Badda',
 'Bimanbandar',
 'Motijheel',
 'Mirpur Model',
 'Mohammadpur',
 'Jatrabari',
 'Ramna',
 'Rampura',
 'Lalbagh',
 'Shah Ali',
 'Shahbagh',
 'Sher-e-Bangla Nagar',
 'Shyampur',
 'Sabujbagh',
 'Sutrapur',
 'Hazaribagh',
 'mirpur',
 'azimpur',
 'mugda',
 'postogola',
 'shyamoli',
 'mohammadpur',
 'moghbazar',
 'maghbazar',
 'mogbazar',
 'chan khar pool',
 'science laboratory',
 'science lab',
 'buet',
 'du',
 'dhaka university',
 'dhaka medical',
 'dmc',
 'shantinagar',
 'meradia',
 'notun bazar',
 'new paltan',
 'noya paltan',
 'technical',
 'gabtoli',
 'motijheel',
 'karwanbazar',
 'farmgate',
 'khamarbari',
 'tejkuni para',
 'purobi',
 'meradia',

In [34]:
eng_lst_areas = [item.lower() for item in eng_lst_areas]

In [30]:
eng_lst_areas = sorted(eng_lst_areas)

In [31]:
eng_lst_areas.remove('chawkbazar model')

In [32]:
eng_lst_areas.extend(['chawkbazar,chalkbazar,chokbazar,chakbazar'])

In [35]:
eng_lst_areas

['adabor',
 'aftab nagar',
 'aftabnagar',
 'airport',
 'azimpur',
 'badda',
 'baily road',
 'banani',
 'bangshal',
 'bimanbandar',
 'buet',
 'cantonment',
 'chan khar pool',
 'dakshinkhan',
 'darus salam',
 'demra',
 'dhaka medical',
 'dhaka university',
 'dhanmondi',
 'dia bari',
 'dmc',
 'du',
 'elephant road',
 'farmgate',
 'gabtoli',
 'gendaria',
 'gulistan',
 'gulshan',
 'hatirpool',
 'hazaribagh',
 'jatrabari',
 'kadamtali',
 'kafrul',
 'kalabagan',
 'kamrangirchar',
 'karwanbazar',
 'khamarbari',
 'khilgaon',
 'khilkhet',
 'kotwali',
 'lalbagh',
 'maghbazar',
 'meradia',
 'meradia',
 'mintu road',
 'mirpur',
 'mirpur model',
 'mogbazar',
 'moghbazar',
 'mohammadpur',
 'mohammadpur',
 'motijheel',
 'motijheel',
 'mugda',
 'new market',
 'new paltan',
 'notun bazar',
 'noya paltan',
 'pallabi',
 'paltan',
 'postogola',
 'puran dhaka',
 'purobi',
 'ramna',
 'rampura',
 'rampura',
 'sabujbagh',
 'sadarghat',
 'savar',
 'science lab',
 'science laboratory',
 'shah ali',
 'shahbagh',


In [36]:
college = [data_lemmatized[i] for i in range(len(data_lemmatized)) if 'college' in data_lemmatized[i]]

In [37]:
college_index = [i for i in range(len(data_lemmatized)) if 'college' in data_lemmatized[i]]

In [38]:
medical = [data_lemmatized[i] for i in range(len(data_lemmatized)) if 'medical' in data_lemmatized[i]]

In [39]:
medical_index = [i for i in range(len(data_lemmatized)) if 'medical' in data_lemmatized[i]]

In [40]:
hospital = [data_lemmatized[i] for i in range(len(data_lemmatized)) if 'hospital' in data_lemmatized[i]]

In [41]:
hospital_index = [i for i in range(len(data_lemmatized)) if 'hospital' in data_lemmatized[i]]

In [42]:
death = [data_lemmatized[i] for i in range(len(data_lemmatized)) if 'death' in data_lemmatized[i]]

In [43]:
death_index = [i for i in range(len(data_lemmatized)) if 'death' in data_lemmatized[i]]

In [44]:
died = [data_lemmatized[i] for i in range(len(data_lemmatized)) if 'died' in data_lemmatized[i]]

In [45]:
died_index = [i for i in range(len(data_lemmatized)) if 'died' in data_lemmatized[i]]

In [46]:
blood = [data_lemmatized[i] for i in range(len(data_lemmatized)) if 'blood' in data_lemmatized[i]]

In [47]:
blood_index = [i for i in range(len(data_lemmatized)) if 'blood' in data_lemmatized[i]]

In [48]:
my_lst = []
places = []
places_index = []

for i in range (len(data_lemmatized)):
    for j in range (len(eng_lst_areas)):
        if eng_lst_areas[j] in data_lemmatized[i]:
            my_lst.append(data_lemmatized[i])
            places.append(eng_lst_areas[j])
            places_index.append(i)
            break

In [49]:
common = list(np.zeros(len(data_lemmatized)))

In [50]:
common = [int(common[i]) for i in range(len(common))]

In [51]:
len(common)

1150

In [52]:
college_posts = common.copy()
for i in range(len(college_index)):
    college_posts[college_index[i]] = 1

In [53]:
medical_posts = common.copy()
for i in range(len(medical_index)):
    medical_posts[medical_index[i]] = 1

In [54]:
hospital_posts = common.copy()
for i in range(len(hospital_index)):
    hospital_posts[hospital_index[i]] = 1

In [55]:
death_posts = common.copy()
for i in range(len(death_index)):
    death_posts[death_index[i]] = 1

In [56]:
died_posts = common.copy()
for i in range(len(died_index)):
    died_posts[died_index[i]] = 1

In [57]:
blood_posts = common.copy()
for i in range(len(blood_index)):
    blood_posts[blood_index[i]] = 1

In [58]:
places_posts = common.copy()
for i in range(len(places_index)):
    places_posts[places_index[i]] = 1

In [59]:
areas = pd.DataFrame(list(zip(college_posts,medical_posts,hospital_posts,death_posts,died_posts,blood_posts,places_posts,data_lemmatized)),columns=['college','medical','hospital','death','died','blood','places','data_lemmatized'])

In [60]:
data_with_areas = pd.concat([eng_data,areas],axis = 1)

In [61]:
data_with_areas

,usernameTweet,ID,text,url,nbr_retweet,nbr_favorite,nbr_reply,datetime,has_media,medias,...,is_retweet,user_id,college,medical,hospital,death,died,blood,places,data_lemmatized
0,DOAWAYWDENGUE,-1.807863e+09,# DENGUEFEVER #AWARENESS - `Dengue may break...,/DOAWAYWDENGUE/status/19666973140,0.0,0.0,0.0,2010-07-27 08:54:07,False,false,...,False,3.354970e+07,0,0,0,0,0,0,0,"[risk, factor, presence, dengue, vector, mosqu..."
1,Crof,5.747630e+08,Bangladesh : 473 #dengue cases in Dhaka,/Crof/status/102929918320586753,0.0,0.0,0.0,2011-08-14 19:29:39,False,false,...,False,1.904299e+07,0,0,1,0,0,0,1,"[emergency, blood_neede, dengue, patient, plac..."
2,DengueNews,-2.682962e+08,# Dengue News: Eradicating dengue menace - Fi...,/DengueNews/status/110057252496482304,0.0,0.0,0.0,2011-09-03 11:31:08,False,false,...,False,3.640113e+08,0,0,0,0,0,0,0,"[threat, denguefever, loom, monsoon, progress,..."
3,Mazedul,-7.115981e+07,"Urgent O- ( O Negative) Blood Needed \n""O-"" ...",/Mazedul/status/137597059736940544,0.0,0.0,0.0,2011-11-18 10:24:30,False,false,...,False,3.761954e+07,0,0,0,0,0,0,0,"[low, socio, economic, zone, dhaka, may, high,..."
4,DengueNews,1.484927e+09,# Dengue News: Dengue-like fever returns stro...,/DengueNews/status/139426347993935872,0.0,0.0,0.0,2011-11-23 11:33:26,False,false,...,False,3.640113e+08,0,0,0,0,0,0,0,"[recent, finding, risk, factor, presence, deng..."
5,DengueNews,-1.891484e+09,# Dengue News: Hydroclimatological variabilit...,/DengueNews/status/194786278334676993,0.0,0.0,0.0,2012-04-24 06:54:02,False,false,...,False,3.640113e+08,0,0,0,0,0,0,0,"[iedcr, report, dengue, case, far]"
6,OnlyinDhaka,-1.048445e+09,"Ah, the smell of #Dhaka 's open #sewerage in ...",/OnlyinDhaka/status/268289421768523776,0.0,0.0,0.0,2012-11-13 01:49:17,False,false,...,False,9.075084e+08,0,0,0,0,0,0,0,"[new, report, circulate, dengue, virus, seroty..."
7,selimbd08,1.170362e+09,"O+ Blood Needed, Loc- Lab-Aid Hospital, Dhan...",/selimbd08/status/271294269006958592,0.0,0.0,0.0,2012-11-21 08:49:28,False,false,...,False,3.521479e+08,0,0,0,0,0,0,0,"[bangladesh, climatic, condition, may, facilit..."
8,IDRC_CRDI,5.496136e+08,« Notre recherche permettra d’établir des donn...,/IDRC_CRDI/status/291987055746117632,0.0,0.0,0.0,2013-01-17 11:15:13,False,false,...,False,4.006517e+07,0,0,0,0,0,0,0,"[aede, larvae, find, house, dhaka, south, http..."
9,IDRC_CRDI,5.579776e+08,« Que savons-nous de la #dengue à Dhaka ? T...,/IDRC_CRDI/status/292023550091595776,1.0,0.0,0.0,2013-01-17 13:40:14,False,false,...,False,4.006517e+07,0,0,0,0,0,0,0,"[aede, larvae, find, house, dhaka, south, deng..."


In [101]:
def countFrequency(my_list): 
  
    freq = {}
    c = 0
    for item in my_list: 
        if (item in freq): 
            freq[item].append(places_index[c])
        else: 
            freq[item] = [places_index[c]]
        c+=1
    return freq

        

freq_dict = countFrequency(places)


for key, value in freq_dict.items(): 
        print (key,' ', value,'\n') 

moghbazar   [1] 

mirpur   [18, 19, 52, 65, 76, 250, 320, 389, 538, 707, 751, 752, 779, 860, 873, 898, 1090] 

gendaria   [40] 

dhanmondi   [49, 50, 1021] 

du   [147] 

tejgaon   [171, 555] 

dmc   [267] 

mugda   [626, 640, 719] 

mohammadpur   [700] 

rampura   [903] 

savar   [923] 



In [102]:
for i in range(len(freq_dict.keys())):
    #list(freq_dict.keys())[i]
    temp_posts = common.copy()
    for j in range(len(freq_dict[list(freq_dict.keys())[i]])):
        temp_posts[freq_dict[list(freq_dict.keys())[i]][j]] = 1
    freq_dict[list(freq_dict.keys())[i]] = temp_posts.copy()

In [106]:
data_with_areas.drop('places',axis=1,inplace=True)

In [108]:
for i in range(len(list(freq_dict.keys()))):
    data_with_areas[list(freq_dict.keys())[i]] = freq_dict[list(freq_dict.keys())[i]]

In [111]:
data_with_areas

,usernameTweet,ID,text,url,nbr_retweet,nbr_favorite,nbr_reply,datetime,has_media,medias,...,mirpur,gendaria,dhanmondi,du,tejgaon,dmc,mugda,mohammadpur,rampura,savar
0,DOAWAYWDENGUE,-1.807863e+09,# DENGUEFEVER #AWARENESS - `Dengue may break...,/DOAWAYWDENGUE/status/19666973140,0.0,0.0,0.0,2010-07-27 08:54:07,False,false,...,0,0,0,0,0,0,0,0,0,0
1,Crof,5.747630e+08,Bangladesh : 473 #dengue cases in Dhaka,/Crof/status/102929918320586753,0.0,0.0,0.0,2011-08-14 19:29:39,False,false,...,0,0,0,0,0,0,0,0,0,0
2,DengueNews,-2.682962e+08,# Dengue News: Eradicating dengue menace - Fi...,/DengueNews/status/110057252496482304,0.0,0.0,0.0,2011-09-03 11:31:08,False,false,...,0,0,0,0,0,0,0,0,0,0
3,Mazedul,-7.115981e+07,"Urgent O- ( O Negative) Blood Needed \n""O-"" ...",/Mazedul/status/137597059736940544,0.0,0.0,0.0,2011-11-18 10:24:30,False,false,...,0,0,0,0,0,0,0,0,0,0
4,DengueNews,1.484927e+09,# Dengue News: Dengue-like fever returns stro...,/DengueNews/status/139426347993935872,0.0,0.0,0.0,2011-11-23 11:33:26,False,false,...,0,0,0,0,0,0,0,0,0,0
5,DengueNews,-1.891484e+09,# Dengue News: Hydroclimatological variabilit...,/DengueNews/status/194786278334676993,0.0,0.0,0.0,2012-04-24 06:54:02,False,false,...,0,0,0,0,0,0,0,0,0,0
6,OnlyinDhaka,-1.048445e+09,"Ah, the smell of #Dhaka 's open #sewerage in ...",/OnlyinDhaka/status/268289421768523776,0.0,0.0,0.0,2012-11-13 01:49:17,False,false,...,0,0,0,0,0,0,0,0,0,0
7,selimbd08,1.170362e+09,"O+ Blood Needed, Loc- Lab-Aid Hospital, Dhan...",/selimbd08/status/271294269006958592,0.0,0.0,0.0,2012-11-21 08:49:28,False,false,...,0,0,0,0,0,0,0,0,0,0
8,IDRC_CRDI,5.496136e+08,« Notre recherche permettra d’établir des donn...,/IDRC_CRDI/status/291987055746117632,0.0,0.0,0.0,2013-01-17 11:15:13,False,false,...,0,0,0,0,0,0,0,0,0,0
9,IDRC_CRDI,5.579776e+08,« Que savons-nous de la #dengue à Dhaka ? T...,/IDRC_CRDI/status/292023550091595776,1.0,0.0,0.0,2013-01-17 13:40:14,False,false,...,0,0,0,0,0,0,0,0,0,0


In [163]:
text_without_area = data_with_areas[(data_with_areas['medical']==0) & (data_with_areas['college']==0) & (data_with_areas['hospital']==0) & (data_with_areas['death']==0) & (data_with_areas['died']==0) & (data_with_areas['blood']==0) & (data_with_areas['places']==0)]['data_lemmatized'].tolist()

In [165]:
text_without_area

[['risk',
  'factor',
  'presence',
  'dengue',
  'vector',
  'mosquitoe',
  'determinant',
  'prevalence',
  'larval',
  'site',
  'selection',
  'dhaka'],
 ['threat',
  'denguefever',
  'loom',
  'monsoon',
  'progress',
  'hit',
  'bangladesh',
  'high',
  'level',
  'rainfall',
  'mosquitoe',
  'hatch',
  'egg',
  'motionless',
  'water',
  'dhaka',
  'city',
  'could',
  'dengue',
  'spread',
  'aedesaegypti'],
 ['low',
  'socio',
  'economic',
  'zone',
  'dhaka',
  'may',
  'high',
  'presence',
  'aedesaegypti',
  'mosquitoe',
  'new',
  'insight'],
 ['recent',
  'finding',
  'risk',
  'factor',
  'presence',
  'dengue',
  'vector',
  'mosquitoe',
  'dhaka'],
 ['iedcr', 'report', 'dengue', 'case', 'far'],
 ['new', 'report', 'circulate', 'dengue', 'virus', 'serotype', 'bangladesh'],
 ['bangladesh',
  'climatic',
  'condition',
  'may',
  'facilitate',
  'vector',
  'bear',
  'disease',
  'include',
  'dengue',
  'new',
  'report',
  'circulate',
  'denguevirus',
  'serotype'],
 

In [112]:
north = [data_lemmatized[i] for i in range(len(data_lemmatized)) if 'north' in data_lemmatized[i]]

In [113]:
north_index = [i for i in range(len(data_lemmatized)) if 'north' in data_lemmatized[i]]

In [115]:
dncc = [data_lemmatized[i] for i in range(len(data_lemmatized)) if 'dncc' in data_lemmatized[i]]

In [116]:
north.extend(dncc)

In [118]:
north_index.extend([i for i in range(len(data_lemmatized)) if 'dncc' in data_lemmatized[i]])

In [119]:
south = [data_lemmatized[i] for i in range(len(data_lemmatized)) if 'south' in data_lemmatized[i]]

In [120]:
south_index = [i for i in range(len(data_lemmatized)) if 'south' in data_lemmatized[i]]

In [121]:
south.extend([data_lemmatized[i] for i in range(len(data_lemmatized)) if 'dscc' in data_lemmatized[i]])

In [122]:
south_index.extend([i for i in range(len(data_lemmatized)) if 'dscc' in data_lemmatized[i]])

In [123]:
north_posts = common.copy()
for i in range(len(north_index)):
    north_posts[north_index[i]] = 1

In [124]:
south_posts = common.copy()
for i in range(len(south_index)):
    south_posts[south_index[i]] = 1

In [133]:
data_with_areas

,usernameTweet,ID,text,url,nbr_retweet,nbr_favorite,nbr_reply,datetime,has_media,medias,...,dhanmondi,du,tejgaon,dmc,mugda,mohammadpur,rampura,savar,north,south
0,DOAWAYWDENGUE,-1.807863e+09,# DENGUEFEVER #AWARENESS - `Dengue may break...,/DOAWAYWDENGUE/status/19666973140,0.0,0.0,0.0,2010-07-27 08:54:07,False,false,...,0,0,0,0,0,0,0,0,0,0
1,Crof,5.747630e+08,Bangladesh : 473 #dengue cases in Dhaka,/Crof/status/102929918320586753,0.0,0.0,0.0,2011-08-14 19:29:39,False,false,...,0,0,0,0,0,0,0,0,0,0
2,DengueNews,-2.682962e+08,# Dengue News: Eradicating dengue menace - Fi...,/DengueNews/status/110057252496482304,0.0,0.0,0.0,2011-09-03 11:31:08,False,false,...,0,0,0,0,0,0,0,0,0,0
3,Mazedul,-7.115981e+07,"Urgent O- ( O Negative) Blood Needed \n""O-"" ...",/Mazedul/status/137597059736940544,0.0,0.0,0.0,2011-11-18 10:24:30,False,false,...,0,0,0,0,0,0,0,0,0,0
4,DengueNews,1.484927e+09,# Dengue News: Dengue-like fever returns stro...,/DengueNews/status/139426347993935872,0.0,0.0,0.0,2011-11-23 11:33:26,False,false,...,0,0,0,0,0,0,0,0,0,0
5,DengueNews,-1.891484e+09,# Dengue News: Hydroclimatological variabilit...,/DengueNews/status/194786278334676993,0.0,0.0,0.0,2012-04-24 06:54:02,False,false,...,0,0,0,0,0,0,0,0,0,0
6,OnlyinDhaka,-1.048445e+09,"Ah, the smell of #Dhaka 's open #sewerage in ...",/OnlyinDhaka/status/268289421768523776,0.0,0.0,0.0,2012-11-13 01:49:17,False,false,...,0,0,0,0,0,0,0,0,0,0
7,selimbd08,1.170362e+09,"O+ Blood Needed, Loc- Lab-Aid Hospital, Dhan...",/selimbd08/status/271294269006958592,0.0,0.0,0.0,2012-11-21 08:49:28,False,false,...,0,0,0,0,0,0,0,0,0,0
8,IDRC_CRDI,5.496136e+08,« Notre recherche permettra d’établir des donn...,/IDRC_CRDI/status/291987055746117632,0.0,0.0,0.0,2013-01-17 11:15:13,False,false,...,0,0,0,0,0,0,0,0,0,1
9,IDRC_CRDI,5.579776e+08,« Que savons-nous de la #dengue à Dhaka ? T...,/IDRC_CRDI/status/292023550091595776,1.0,0.0,0.0,2013-01-17 13:40:14,False,false,...,0,0,0,0,0,0,0,0,0,1


In [130]:
data_with_areas['north'] = north_posts
data_with_areas['south'] = south_posts

In [132]:
data_with_areas.columns

Index(['usernameTweet', 'ID', 'text', 'url', 'nbr_retweet', 'nbr_favorite',
       'nbr_reply', 'datetime', 'has_media', 'medias', 'is_reply',
       'is_retweet', 'user_id', 'college', 'medical', 'hospital', 'death',
       'died', 'blood', 'data_lemmatized', 'moghbazar', 'mirpur', 'gendaria',
       'dhanmondi', 'du', 'tejgaon', 'dmc', 'mugda', 'mohammadpur', 'rampura',
       'savar', 'north', 'south'],
      dtype='object')

In [166]:
text_without_area = data_with_areas[(data_with_areas['medical']==0) & (data_with_areas['college']==0) & (data_with_areas['hospital']==0) & (data_with_areas['death']==0) & (data_with_areas['died']==0) & (data_with_areas['blood']==0) & (data_with_areas['places']==0) & (data_with_areas['north']==0) & (data_with_areas['south']==0)]['data_lemmatized'].tolist()

In [169]:
text_without_area

[['risk',
  'factor',
  'presence',
  'dengue',
  'vector',
  'mosquitoe',
  'determinant',
  'prevalence',
  'larval',
  'site',
  'selection',
  'dhaka'],
 ['threat',
  'denguefever',
  'loom',
  'monsoon',
  'progress',
  'hit',
  'bangladesh',
  'high',
  'level',
  'rainfall',
  'mosquitoe',
  'hatch',
  'egg',
  'motionless',
  'water',
  'dhaka',
  'city',
  'could',
  'dengue',
  'spread',
  'aedesaegypti'],
 ['low',
  'socio',
  'economic',
  'zone',
  'dhaka',
  'may',
  'high',
  'presence',
  'aedesaegypti',
  'mosquitoe',
  'new',
  'insight'],
 ['recent',
  'finding',
  'risk',
  'factor',
  'presence',
  'dengue',
  'vector',
  'mosquitoe',
  'dhaka'],
 ['iedcr', 'report', 'dengue', 'case', 'far'],
 ['new', 'report', 'circulate', 'dengue', 'virus', 'serotype', 'bangladesh'],
 ['bangladesh',
  'climatic',
  'condition',
  'may',
  'facilitate',
  'vector',
  'bear',
  'disease',
  'include',
  'dengue',
  'new',
  'report',
  'circulate',
  'denguevirus',
  'serotype'],
 

In [143]:
medical_index.extend([i for i in range(len(data_lemmatized)) if 'ssmc' in data_lemmatized[i]])

In [147]:
medical_posts[medical_index[len(medical_index)-1]] = 1

In [148]:
data_with_areas['medical'] = medical_posts

In [149]:
cities = pd.read_csv('haha1.csv' , sep = '\t')

In [150]:
cities = list(cities['Name'])

In [151]:
cities = [cities[i].lower() for i in range(len(cities))]

In [152]:
cities.remove('dhaka')

In [155]:
my_lst = []
places = []
places_index = []

for i in range (len(data_lemmatized)):
    for j in range (len(cities)):
        if cities[j] in data_lemmatized[i]:
            my_lst.append(data_lemmatized[i])
            places.append(cities[j])
            places_index.append(i)
            break

In [168]:
def countFrequency(my_list): 
  
    freq = {}
    c = 0
    for item in my_list: 
        if (item in freq): 
            freq[item].append(places_index[c])
        else: 
            freq[item] = [places_index[c]]
        c+=1
    return freq

        

freq_dict = countFrequency(places)


for key, value in freq_dict.items(): 
        print (key,' ', value,'\n') 

khulna   [28, 182, 216, 261, 273, 543, 733, 735, 736, 748, 778, 792, 814, 818, 897, 899, 937, 973, 995, 1001] 

habiganj   [205] 

bogra   [249, 317, 398, 827] 

sylhet   [324] 

barisal   [327, 329] 

faridpur   [516, 785, 790] 

rangpur   [594, 595, 600, 614, 889, 890, 907, 915] 

dinajpur   [701] 

tangail   [704] 

mymensingh   [719, 799, 800, 865, 866, 867, 870, 988, 990] 

satkhira   [855, 992] 

madaripur   [872, 874] 

chittagong   [920, 1113, 1115, 1143] 

kushtia   [987] 

jessore   [997] 

bandarban   [1006] 



In [169]:
for i in range(len(freq_dict.keys())):
    #list(freq_dict.keys())[i]
    temp_posts = common.copy()
    for j in range(len(freq_dict[list(freq_dict.keys())[i]])):
        temp_posts[freq_dict[list(freq_dict.keys())[i]][j]] = 1
    freq_dict[list(freq_dict.keys())[i]] = temp_posts.copy()

In [170]:
freq_dict

{'khulna': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [171]:
for i in range(len(list(freq_dict.keys()))):
    data_with_areas[list(freq_dict.keys())[i]] = freq_dict[list(freq_dict.keys())[i]]

In [172]:
data_with_areas

,usernameTweet,ID,text,url,nbr_retweet,nbr_favorite,nbr_reply,datetime,has_media,medias,...,rangpur,dinajpur,tangail,mymensingh,satkhira,madaripur,chittagong,kushtia,jessore,bandarban
0,DOAWAYWDENGUE,-1.807863e+09,# DENGUEFEVER #AWARENESS - `Dengue may break...,/DOAWAYWDENGUE/status/19666973140,0.0,0.0,0.0,2010-07-27 08:54:07,False,false,...,0,0,0,0,0,0,0,0,0,0
1,Crof,5.747630e+08,Bangladesh : 473 #dengue cases in Dhaka,/Crof/status/102929918320586753,0.0,0.0,0.0,2011-08-14 19:29:39,False,false,...,0,0,0,0,0,0,0,0,0,0
2,DengueNews,-2.682962e+08,# Dengue News: Eradicating dengue menace - Fi...,/DengueNews/status/110057252496482304,0.0,0.0,0.0,2011-09-03 11:31:08,False,false,...,0,0,0,0,0,0,0,0,0,0
3,Mazedul,-7.115981e+07,"Urgent O- ( O Negative) Blood Needed \n""O-"" ...",/Mazedul/status/137597059736940544,0.0,0.0,0.0,2011-11-18 10:24:30,False,false,...,0,0,0,0,0,0,0,0,0,0
4,DengueNews,1.484927e+09,# Dengue News: Dengue-like fever returns stro...,/DengueNews/status/139426347993935872,0.0,0.0,0.0,2011-11-23 11:33:26,False,false,...,0,0,0,0,0,0,0,0,0,0
5,DengueNews,-1.891484e+09,# Dengue News: Hydroclimatological variabilit...,/DengueNews/status/194786278334676993,0.0,0.0,0.0,2012-04-24 06:54:02,False,false,...,0,0,0,0,0,0,0,0,0,0
6,OnlyinDhaka,-1.048445e+09,"Ah, the smell of #Dhaka 's open #sewerage in ...",/OnlyinDhaka/status/268289421768523776,0.0,0.0,0.0,2012-11-13 01:49:17,False,false,...,0,0,0,0,0,0,0,0,0,0
7,selimbd08,1.170362e+09,"O+ Blood Needed, Loc- Lab-Aid Hospital, Dhan...",/selimbd08/status/271294269006958592,0.0,0.0,0.0,2012-11-21 08:49:28,False,false,...,0,0,0,0,0,0,0,0,0,0
8,IDRC_CRDI,5.496136e+08,« Notre recherche permettra d’établir des donn...,/IDRC_CRDI/status/291987055746117632,0.0,0.0,0.0,2013-01-17 11:15:13,False,false,...,0,0,0,0,0,0,0,0,0,0
9,IDRC_CRDI,5.579776e+08,« Que savons-nous de la #dengue à Dhaka ? T...,/IDRC_CRDI/status/292023550091595776,1.0,0.0,0.0,2013-01-17 13:40:14,False,false,...,0,0,0,0,0,0,0,0,0,0


In [178]:
data_with_areas[list(data_with_areas.columns[13:])]

,college,medical,hospital,death,died,blood,data_lemmatized,moghbazar,mirpur,gendaria,...,rangpur,dinajpur,tangail,mymensingh,satkhira,madaripur,chittagong,kushtia,jessore,bandarban
0,0,0,0,0,0,0,"[risk, factor, presence, dengue, vector, mosqu...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,"[emergency, blood_neede, dengue, patient, plac...",1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,"[threat, denguefever, loom, monsoon, progress,...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,"[low, socio, economic, zone, dhaka, may, high,...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,"[recent, finding, risk, factor, presence, deng...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,"[iedcr, report, dengue, case, far]",0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,"[new, report, circulate, dengue, virus, seroty...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,"[bangladesh, climatic, condition, may, facilit...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,"[aede, larvae, find, house, dhaka, south, http...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,"[aede, larvae, find, house, dhaka, south, deng...",0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [180]:
affected = [data_lemmatized[i] for i in range(len(data_lemmatized)) if 'affected' in data_lemmatized[i]]

In [181]:
affected_index = [i for i in range(len(data_lemmatized)) if 'affected' in data_lemmatized[i]]

In [182]:
affected_posts = common.copy()
for i in range(len(affected_index)):
    affected_posts[affected_index[i]] = 1

In [183]:
data_with_areas['affected'] = affected_posts

In [210]:
data_with_areas.columns

Index(['usernameTweet', 'ID', 'text', 'url', 'nbr_retweet', 'nbr_favorite',
       'nbr_reply', 'datetime', 'has_media', 'medias', 'is_reply',
       'is_retweet', 'user_id', 'college', 'medical', 'hospital', 'death',
       'died', 'blood', 'data_lemmatized', 'moghbazar', 'mirpur', 'gendaria',
       'dhanmondi', 'du', 'tejgaon', 'dmc', 'mugda', 'mohammadpur', 'rampura',
       'savar', 'north', 'south', 'khulna', 'habiganj', 'bogra', 'sylhet',
       'barisal', 'faridpur', 'rangpur', 'dinajpur', 'tangail', 'mymensingh',
       'satkhira', 'madaripur', 'chittagong', 'kushtia', 'jessore',
       'bandarban', 'affected', 'dhaka', 'cmh'],
      dtype='object')

In [233]:
series_without = data_with_areas[(data_with_areas['north']==0) & (data_with_areas['south']==0) & (data_with_areas['moghbazar']==0) &
                (data_with_areas['mirpur']==0) & (data_with_areas['gendaria']==0) & (data_with_areas['dhanmondi']==0) &
                (data_with_areas['du']==0) & (data_with_areas['tejgaon']==0) & (data_with_areas['south']==0) & 
               (data_with_areas['dmc']==0) & (data_with_areas['mugda']==0) & (data_with_areas['mohammadpur']==0) & 
               (data_with_areas['rampura']==0) & (data_with_areas['savar']==0) & (data_with_areas['khulna']==0) & 
               (data_with_areas['habiganj']==0) & (data_with_areas['bogra']==0) & (data_with_areas['sylhet']==0) & 
               (data_with_areas['barisal']==0) & (data_with_areas['faridpur']==0) & (data_with_areas['rangpur']==0) & 
               (data_with_areas['dinajpur']==0) & (data_with_areas['tangail']==0) & (data_with_areas['mymensingh']==0) & 
               (data_with_areas['satkhira']==0) & (data_with_areas['madaripur']==0) & (data_with_areas['chittagong']==0) & 
               (data_with_areas['kushtia']==0) & (data_with_areas['jessore']==0) & (data_with_areas['bandarban']==0) &
                (data_with_areas['cmh']==0) & (data_with_areas['university']==0)& (data_with_areas['medical']==0) &
                (data_with_areas['college']==0) & (data_with_areas['hospital']==0) & (data_with_areas['blood']==0) &
                (data_with_areas['died']==0) ]['data_lemmatized']

In [234]:
dhaka_index = [list(series_without.index)[i] for i in range(len(series_without)) if 'dhaka' in list(series_without)[i]]

In [235]:
dhaka = [list(series_without)[i] for i in range(len(series_without)) if 'dhaka' in list(series_without)[i]]

In [236]:
dhaka_posts = common.copy()
for i in range(len(dhaka_index)):
    dhaka_posts[dhaka_index[i]] = 1

In [237]:
data_with_areas['dhaka'] = dhaka_posts

In [238]:
dhaka

[['risk',
  'factor',
  'presence',
  'dengue',
  'vector',
  'mosquitoe',
  'determinant',
  'prevalence',
  'larval',
  'site',
  'selection',
  'dhaka'],
 ['threat',
  'denguefever',
  'loom',
  'monsoon',
  'progress',
  'hit',
  'bangladesh',
  'high',
  'level',
  'rainfall',
  'mosquitoe',
  'hatch',
  'egg',
  'motionless',
  'water',
  'dhaka',
  'city',
  'could',
  'dengue',
  'spread',
  'aedesaegypti'],
 ['low',
  'socio',
  'economic',
  'zone',
  'dhaka',
  'may',
  'high',
  'presence',
  'aedesaegypti',
  'mosquitoe',
  'new',
  'insight'],
 ['recent',
  'finding',
  'risk',
  'factor',
  'presence',
  'dengue',
  'vector',
  'mosquitoe',
  'dhaka'],
 ['case',
  'dengue',
  'report',
  'far',
  'year',
  'bangladesh',
  'result',
  'death',
  'people',
  'capital',
  'dhaka'],
 ['dhaka', 'risk', 'dengue', 'outbreak'],
 ['construction',
  'site',
  'garbage',
  'dhaka',
  'say',
  'create',
  'aede',
  'breeding',
  'ground'],
 ['mosquitoe',
  'hatch',
  'egg',
  'motio

In [239]:
len(dhaka)

164

In [240]:
len(series_without)

862

In [206]:
cmh = [data_lemmatized[i] for i in range(len(data_lemmatized)) if 'cmh' in data_lemmatized[i]]

In [207]:
cmh_index = [i for i in range(len(data_lemmatized)) if 'university' in data_lemmatized[i]]

In [208]:
cmh_posts = common.copy()
for i in range(len(cmh_index)):
    cmh_posts[cmh_index[i]] = 1

In [209]:
data_with_areas['cmh'] = cmh_posts

In [218]:
university = [data_lemmatized[i] for i in range(len(data_lemmatized)) if 'university' in data_lemmatized[i]]
university_index = [i for i in range(len(data_lemmatized)) if 'university' in data_lemmatized[i]]

In [219]:
university_posts = common.copy()
for i in range(len(university_index)):
    university_posts[university_index[i]] = 1

In [220]:
data_with_areas['university'] = university_posts

In [229]:
medical.extend([data_lemmatized[i] for i in range(len(data_lemmatized)) if 'medical_college' in data_lemmatized[i]])

In [230]:
medical_index.extend([i for i in range(len(data_lemmatized)) if 'medical_college' in data_lemmatized[i]])

In [231]:
medical_posts = common.copy()
for i in range(len(medical_index)):
    medical_posts[medical_index[i]] = 1

In [232]:
data_with_areas['medical'] = medical_posts

In [243]:
export_excel = data_with_areas.to_excel(r'C:\Users\Alvi Rahman\Desktop\Farhana Miss\updated_english.xlsx')

In [246]:
optimal_model = [(0,
  '0.250*"twitt" + 0.234*"pic" + 0.039*"awareness" + 0.022*"continue" + '
  '0.018*"support" + 0.016*"clean" + 0.016*"service" + 0.015*"malaria" + '
  '0.013*"volunteer" + 0.012*"raise"'),
 (1,
  '0.242*"dhaka" + 0.085*"mosquitoe" + 0.063*"city" + 0.038*"chikungunya" + '
  '0.029*"risk" + 0.022*"research" + 0.013*"rain" + 0.013*"burden" + '
  '0.011*"level" + 0.011*"water"'),
 (2,
  '0.185*"dhaka" + 0.062*"blood" + 0.041*"contact" + 0.027*"suffer" + '
  '0.026*"south" + 0.026*"blood_needed" + 0.024*"bag" + 0.024*"urgent" + '
  '0.019*"platelet" + 0.018*"facebook"'),
 (3,
  '0.254*"fever" + 0.166*"health" + 0.083*"www" + 0.032*"department" + '
  '0.027*"pic" + 0.025*"confirm" + 0.017*"twitter" + 0.014*"majority" + '
  '0.013*"confirmed_html" + 0.013*"call"'),
 (4,
  '0.133*"disease" + 0.105*"dengue" + 0.071*"spread" + 0.063*"mosquito_borne" '
  '+ 0.037*"infection" + 0.026*"district" + 0.025*"virus" + 0.022*"world" + '
  '0.020*"viral" + 0.017*"protect"'),
 (5,
  '0.247*"dengue" + 0.096*"epidemic" + 0.028*"order" + 0.023*"due" + '
  '0.020*"doh" + 0.020*"reach" + 0.019*"hello_professional" + '
  '0.019*"children_book" + 0.019*"gig" + 0.019*"please_check"'),
 (6,
  '0.503*"dengue" + 0.025*"time" + 0.018*"stop" + 0.011*"set" + '
  '0.010*"shar_es" + 0.010*"girl" + 0.007*"eidulazha" + 0.007*"transmission" + '
  '0.007*"simple" + 0.007*"urge"'),
 (7,
  '0.282*"dengue" + 0.054*"day" + 0.040*"doctor" + 0.037*"symptom" + '
  '0.034*"prevention" + 0.028*"life" + 0.020*"severe" + 0.018*"make" + '
  '0.014*"eid" + 0.013*"bone"'),
 (8,
  '0.280*"bangladesh" + 0.156*"dengue" + 0.063*"denguefever" + 0.049*"rise" + '
  '0.029*"capital" + 0.018*"hospitalize" + 0.013*"menace" + '
  '0.010*"sheikhhasina" + 0.010*"colleague" + 0.010*"population"'),
 (9,
  '0.172*"news" + 0.157*"bangladesh" + 0.116*"hour" + 0.095*"dengue" + '
  '0.041*"hospitalise" + 0.039*"threat" + 0.029*"find" + 0.025*"multi_organ" + '
  '0.018*"hrs" + 0.017*"dead"'),
 (10,
  '0.222*"dengue" + 0.123*"bangladesh" + 0.067*"http" + 0.037*"minister" + '
  '0.037*"official" + 0.021*"article" + 0.021*"claim" + 0.018*"home" + '
  '0.013*"ward" + 0.012*"twitt"'),
 (11,
  '0.241*"hospital" + 0.191*"patient" + 0.063*"number" + 0.034*"admit" + '
  '0.023*"total" + 0.022*"mirpur" + 0.019*"emergency" + 0.016*"friday" + '
  '0.013*"place" + 0.013*"give"'),
 (12,
  '0.185*"mosquito" + 0.058*"aede" + 0.058*"control" + 0.035*"fight" + '
  '0.026*"city_corporation" + 0.026*"prevent" + 0.019*"mayor" + 0.019*"area" + '
  '0.018*"technique" + 0.018*"effective"'),
 (13,
  '0.235*"outbreak" + 0.161*"bangladesh" + 0.070*"bad" + 0.043*"diagnose" + '
  '0.029*"struggle" + 0.029*"record" + 0.023*"hit" + 0.018*"worst_ever" + '
  '0.017*"face" + 0.015*"overwhelm"'),
 (14,
  '0.358*"dengue" + 0.032*"increase" + 0.021*"test" + 0.017*"stay" + '
  '0.016*"safe" + 0.013*"caso" + 0.012*"count" + 0.012*"status" + 0.011*"good" '
  '+ 0.011*"team"'),
 (15,
  '0.144*"die" + 0.078*"https" + 0.062*"thedailystar_net" + 0.060*"fever" + '
  '0.054*"country" + 0.039*"medical_college" + 0.035*"treatment" + 0.034*"www" '
  '+ 0.029*"khulna" + 0.027*"include"'),
 (16,
  '0.159*"people" + 0.117*"bangladesh" + 0.069*"country" + 0.038*"govt" + '
  '0.037*"child" + 0.035*"infect" + 0.024*"read" + 0.023*"tackle" + '
  '0.023*"work" + 0.021*"situation"'),
 (17,
  '0.190*"case" + 0.144*"year" + 0.085*"report" + 0.067*"death" + '
  '0.034*"deadly" + 0.034*"high" + 0.027*"record" + 0.022*"dgh" + '
  '0.018*"month" + 0.018*"july"'),
 (18,
  '0.233*"https" + 0.162*"bangladesh" + 0.064*"www" + 0.039*"dengueoutbreak" + '
  '0.025*"bit_ly" + 0.020*"dhakatribune" + 0.016*"toll" + 0.012*"politic" + '
  '0.011*"het" + 0.011*"bf"'),
 (19,
  '0.135*"health" + 0.036*"healthcare" + 0.035*"medical" + 0.035*"government" '
  '+ 0.030*"family" + 0.023*"issue" + 0.020*"vaccine" + 0.020*"accord" + '
  '0.017*"net" + 0.017*"crisis"')]

In [247]:
new_topics = [row[1] for row in lda_model.print_topics()]

AttributeError: 'tuple' object has no attribute 'split'